In [1]:
from trader.binance_spot_trader import BinanceSpotTrader
from gateway.binance_future import Interval
from utils.config import signal_data
import time
import logging
from trader.binance_spot_trader import BinanceSpotTrader
from trader.binance_future_trader import BinanceFutureTrader
from utils import config
from apscheduler.schedulers.background import BackgroundScheduler
format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO, format=format, filename='log.txt')
logging.getLogger("apscheduler.scheduler").setLevel(logging.WARNING)
logger = logging.getLogger('binance')
from typing import Union
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import datetime

trader = BinanceSpotTrader()


In [4]:
def get_rate(symbol):
                klines = trader.get_klines(symbol=symbol, interval='1h', limit=1000)
                df = pd.DataFrame(klines, dtype=np.float64,
                                columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'turnover', 'a2',
                                        'a3', 'a4', 'a5'])
                df = df[['open_time', 'open', 'high', 'low', 'close', 'volume', 'turnover']]
                df.set_index('open_time', inplace=True)
                df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=8)
                pct = df['close']/df['open']
                return pct

In [ ]:
def calalpha(X_train,y_train):
    from sklearn.linear_model import LinearRegression
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)

In [ ]:
Dict_rate = {'ETHUSDT':get_rate('ETHUSDT'),'KNCUSDT':get_rate('KNCUSDT')}



In [3]:
def get_data(trader):
    # traders.symbols is a dict data structure.
    symbols = trader.symbols_dict.keys()
    signals = []
   # print(symbols)
    # we calculate the signal here.
    for symbol in symbols:
        print(symbol)
        klines = trader.get_klines(symbol=symbol, interval='15m', limit=100)
        if len(klines) > 0:
            df = pd.DataFrame(klines, dtype=np.float64,
                              columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'turnover', 'a2',
                                       'a3', 'a4', 'a5'])
            df = df[['open_time', 'open', 'high', 'low', 'close', 'volume', 'turnover']]
            df.set_index('open_time', inplace=True)
            df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=8)

            df_4hour = df.resample(rule='1H').agg({'open': 'first',
                                        'high': 'max',
                                        'low': 'min',
                                        'close': 'last',
                                        'volume': 'sum',
                                        'turnover': 'sum'
                                        })

            # print(df)

            # calculate the pair's price change is one hour. you can modify the code below.
            pct = df['close'] / df['open'] - 1
            pct_4h = df_4hour['close']/df_4hour['open'] - 1

            value = {'pct': pct[-1], 'pct_4h':pct_4h[-1] , 'symbol': symbol, 'hour_turnover': df['turnover'][-1]}


            # calculate your signal here.
            if value['pct'] >= config.pump_pct or value['pct_4h'] >= config.pump_pct_4h:
                # the signal 1 mean buy signal.
                value['signal'] = 1
            elif value['pct'] <= -config.pump_pct or value['pct_4h'] <= -config.pump_pct_4h:
                value['signal'] = -1
            else:
                value['signal'] = 0

            signals.append(value)
    print(signals)       
    signals.sort(key=lambda x: x['pct'], reverse=True)
    signal_data['id'] = signal_data['id'] + 1
    signal_data['time'] = datetime.now()
    signal_data['signals'] = signals
    print(signal_data)

In [4]:
trader.get_exchange_info()
get_data(trader)

BTCUSDT
ETHUSDT
BNBUSDT
NEOUSDT
LTCUSDT
QTUMUSDT
ADAUSDT
XRPUSDT
EOSUSDT
TUSDUSDT
IOTAUSDT
XLMUSDT
ONTUSDT
TRXUSDT
ETCUSDT
ICXUSDT
NULSUSDT
VETUSDT
USDCUSDT
LINKUSDT
WAVESUSDT
ONGUSDT
HOTUSDT
ZILUSDT
ZRXUSDT
FETUSDT
BATUSDT
XMRUSDT
ZECUSDT
IOSTUSDT
CELRUSDT
DASHUSDT
OMGUSDT
THETAUSDT
ENJUSDT
MITHUSDT
MATICUSDT
ATOMUSDT
TFUELUSDT
ONEUSDT
FTMUSDT
ALGOUSDT
GTOUSDT
DOGEUSDT
DUSKUSDT
ANKRUSDT
WINUSDT
COSUSDT
COCOSUSDT
MTLUSDT
TOMOUSDT
PERLUSDT
DENTUSDT
MFTUSDT
KEYUSDT
DOCKUSDT
WANUSDT
FUNUSDT
CVCUSDT
CHZUSDT
BANDUSDT
BUSDUSDT
BEAMUSDT
XTZUSDT
RENUSDT
RVNUSDT
HBARUSDT
NKNUSDT
STXUSDT
KAVAUSDT
ARPAUSDT
IOTXUSDT
RLCUSDT
CTXCUSDT
BCHUSDT
TROYUSDT
VITEUSDT
FTTUSDT
EURUSDT
OGNUSDT
DREPUSDT
TCTUSDT
WRXUSDT
BTSUSDT
LSKUSDT
BNTUSDT
LTOUSDT
AIONUSDT
MBLUSDT
COTIUSDT
STPTUSDT
WTCUSDT
DATAUSDT
SOLUSDT
CTSIUSDT
HIVEUSDT
CHRUSDT
GXSUSDT
ARDRUSDT
MDTUSDT
STMXUSDT
KNCUSDT
REPUSDT
LRCUSDT
PNTUSDT
COMPUSDT
SCUSDT
ZENUSDT
SNXUSDT
VTHOUSDT
DGBUSDT
GBPUSDT
SXPUSDT
MKRUSDT
DCRUSDT
STORJUSDT
MANAUSDT
AUDUSDT
YFIU

In [5]:
x= signal_data.get('signals', [])
print(x[0:10])
pctm = []
pctm2=[]
for signal in x:
            s = signal['symbol']
            if signal['signal'] == 1 and signal['hour_turnover'] >= config.turnover_threshold:
                print(s," ok",type(signal['hour_turnover']))
                pctm.append(signal)
              #  pctm2.append(signal)
print(np.max(pctm),pctm)

[{'pct': 0.08656957928802589, 'pct_4h': 0.32668181369159344, 'symbol': 'ALPINEUSDT', 'hour_turnover': 10225691.841264, 'signal': 1}, {'pct': 0.0208098987626546, 'pct_4h': 0.03242320819112621, 'symbol': 'ARUSDT', 'hour_turnover': 410362.0622, 'signal': 0}, {'pct': 0.019857624578493827, 'pct_4h': -0.061379310344827576, 'symbol': 'CELOUSDT', 'hour_turnover': 365833.2165, 'signal': 0}, {'pct': 0.018112488083889433, 'pct_4h': 0.0009372071227742307, 'symbol': 'LSKUSDT', 'hour_turnover': 22094.5376, 'signal': 0}, {'pct': 0.01740294511378848, 'pct_4h': -0.07486305538648808, 'symbol': 'IMXUSDT', 'hour_turnover': 509028.51585, 'signal': 0}, {'pct': 0.016577403723539996, 'pct_4h': -0.008950770760815518, 'symbol': 'RENUSDT', 'hour_turnover': 132725.0695, 'signal': 0}, {'pct': 0.015026935072299352, 'pct_4h': -0.042780748663101664, 'symbol': 'FIROUSDT', 'hour_turnover': 38063.2288, 'signal': 0}, {'pct': 0.014925373134328401, 'pct_4h': -0.040752351097178674, 'symbol': 'PEOPLEUSDT', 'hour_turnover': 1

1000000


In [ ]:
from utils import config
config.loads('./config.json')
print(config.turnover_threshold)

In [6]:
trader.start()

Positons: dict_keys(['GMTUSDT'])
买入信号 {'id': 1, 'signals': [{'pct': 0.08656957928802589, 'pct_4h': 0.32668181369159344, 'symbol': 'ALPINEUSDT', 'hour_turnover': 10225691.841264, 'signal': 1}, {'pct': 0.0208098987626546, 'pct_4h': 0.03242320819112621, 'symbol': 'ARUSDT', 'hour_turnover': 410362.0622, 'signal': 0}, {'pct': 0.019857624578493827, 'pct_4h': -0.061379310344827576, 'symbol': 'CELOUSDT', 'hour_turnover': 365833.2165, 'signal': 0}, {'pct': 0.018112488083889433, 'pct_4h': 0.0009372071227742307, 'symbol': 'LSKUSDT', 'hour_turnover': 22094.5376, 'signal': 0}, {'pct': 0.01740294511378848, 'pct_4h': -0.07486305538648808, 'symbol': 'IMXUSDT', 'hour_turnover': 509028.51585, 'signal': 0}, {'pct': 0.016577403723539996, 'pct_4h': -0.008950770760815518, 'symbol': 'RENUSDT', 'hour_turnover': 132725.0695, 'signal': 0}, {'pct': 0.015026935072299352, 'pct_4h': -0.042780748663101664, 'symbol': 'FIROUSDT', 'hour_turnover': 38063.2288, 'signal': 0}, {'pct': 0.014925373134328401, 'pct_4h': -0.040

In [ ]:
trader.get_klines(symbol="ETHUSDT",interval=Interval.HOUR_1,limit=100)